# 01. Market Expectations In The Cross-Section Of Present Values

## Imports

In [ ]:
import numpy as np
import pandas as pd
import sys
import os

from matplotlib import pyplot as plt
from settings import config
from pathlib import Path
import load_data
import regressions
import pull_CRSP_index
import pull_ken_french_data
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web

# Load environment variables
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

## Prep Data

### Step 1: Fama-French Data

In [ ]:
get_available_datasets()

#### Pulling Ken French data

In [ ]:
bm_6_portfolios = pull_ken_french_data.pull_ken_french_excel(dataset_name="6_Portfolios_2x3",
     data_dir=DATA_DIR,
     log=True,
     start_date=START_DATE,
     end_date=END_DATE)

bm_25_portfolios = pull_ken_french_data.pull_ken_french_excel(dataset_name="25_Portfolios_5x5",
     data_dir=DATA_DIR,
     log=True,
     start_date=START_DATE,
     end_date=END_DATE)

bm_100_portfolios = pull_ken_french_data.pull_ken_french_excel(
     dataset_name="100_Portfolios_10x10",
     data_dir=DATA_DIR,
     log=True,
     start_date=START_DATE,
     end_date=END_DATE)

In [ ]:
bm_6_portfolios = load_data.load_ken_french()
bm_6_portfolios

### Step 2: CRSP Market Capitalization Weighted Index Data

In [ ]:
crsp = pull_CRSP_index.pull_crsp_value_weighted_index()

In [ ]:
crsp_loaded = load_data.load_crsp_index()
crsp_loaded

### Step 3: Excess Returns

In [ ]:
ex_ret = load_data.load_and_compute_log_returns()
ex_ret

# Run Regressions for B/M ratios

### 6 Portfolios
#### Monthly Regressions

In [ ]:
def regress():
    portfolio_datasets = {
        "6-Portfolios": "6_Portfolios_2x3"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\nProcessing {label} Data")
        in_sample = regressions.run_in_sample_pls(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            end_date='1980-01-01')
        recursive = regressions.run_recursive_forecast(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            start_train_date='1930-01-01', 
            end_train_date='1980-01-01',
            end_forecast_date='2011-01-01')
        regressions.display_results(label, in_sample, recursive)

        results[label] = {
            "R2 In-Sample": in_sample["third_model"].rsquared,  
            "R2 Out-of-Sample": recursive["R2_oos"]
        }

    return results

# Run function and extract summary DataFrame
results = regress()
summary_df_6_monthly = pd.DataFrame(results).T 

# Display the DataFrame
summary_df_6_monthly

In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_6_monthly.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_6_monthly.to_latex(summary_tex_path, index=True, float_format="%.6f")

### 6 Portfolios
#### Annual Regressions

In [ ]:
def regress_annual():
    portfolio_datasets = {
        "6-Portfolios": "6_Portfolios_2x3"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\n[Annual] Processing {label} Data")
        in_sample_results = regressions.run_in_sample_pls_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            end_date='1980-01-01'
        )
        forecast_series, actual_series, R2_oos = regressions.run_recursive_forecast_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            start_train_year=1930,
            end_train_year=1980,
            end_forecast_year=2010,
            n_components=1
        )
        regressions.display_results_annual(label, in_sample_results, forecast_series, actual_series, R2_oos)

        results[label] = {
            "R2 In-Sample": in_sample_results["third_model"].rsquared,  
            "R2 Out-of-Sample": R2_oos
        }
    
    return results

# Run function and extract summary DataFrame
results_annual = regress_annual()
summary_df_6_annual = pd.DataFrame(results_annual).T

# Display the DataFrame
summary_df_6_annual


In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_6_annual.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_6_annual.to_latex(summary_tex_path, index=True, float_format="%.6f")

### 25 Portfolios
#### Monthly Regressions

In [ ]:
def regress():
    portfolio_datasets = {
        "25-Portfolios": "25_Portfolios_5x5"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\nProcessing {label} Data")
        in_sample = regressions.run_in_sample_pls(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            end_date='1980-01-01')
        recursive = regressions.run_recursive_forecast(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            start_train_date='1930-01-01', 
            end_train_date='1980-01-01',
            end_forecast_date='2011-01-01')
        regressions.display_results(label, in_sample, recursive)

        results[label] = {
            "R2 In-Sample": in_sample["third_model"].rsquared,  
            "R2 Out-of-Sample": recursive["R2_oos"]
        }

    return results

# Run function and extract summary DataFrame
results = regress()
summary_df_25_monthly = pd.DataFrame(results).T 

# Display the DataFrame
summary_df_25_monthly

In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_25_monthly.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_25_monthly.to_latex(summary_tex_path, index=True, float_format="%.6f")

### 25 Portfolios
#### Annual Regressions

In [ ]:
def regress_annual():
    portfolio_datasets = {
        "25-Portfolios": "25_Portfolios_5x5"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\n[Annual] Processing {label} Data")
        in_sample_results = regressions.run_in_sample_pls_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            end_date='1980-01-01'
        )
        forecast_series, actual_series, R2_oos = regressions.run_recursive_forecast_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            start_train_year=1930,
            end_train_year=1980,
            end_forecast_year=2010,
            n_components=1
        )
        regressions.display_results_annual(label, in_sample_results, forecast_series, actual_series, R2_oos)

        results[label] = {
            "R2 In-Sample": in_sample_results["third_model"].rsquared,  
            "R2 Out-of-Sample": R2_oos
        }
    
    return results

# Run function and extract summary DataFrame
results_annual = regress_annual()
summary_df_25_annual = pd.DataFrame(results_annual).T

# Display the DataFrame
summary_df_25_annual

In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_25_annual.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_25_annual.to_latex(summary_tex_path, index=True, float_format="%.6f")

### 100 Portfolios
#### Monthly Regressions

In [ ]:
def regress():
    portfolio_datasets = {
        "100-Portfolios": "100_Portfolios_10x10"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\nProcessing {label} Data")
        in_sample = regressions.run_in_sample_pls(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            end_date='1980-01-01')
        recursive = regressions.run_recursive_forecast(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            start_train_date='1930-01-01', 
            end_train_date='1980-01-01',
            end_forecast_date='2011-01-01')
        regressions.display_results(label, in_sample, recursive)

        results[label] = {
            "R2 In-Sample": in_sample["third_model"].rsquared,  
            "R2 Out-of-Sample": recursive["R2_oos"]
        }

    return results

# Run function and extract summary DataFrame
results = regress()
summary_df_100_monthly = pd.DataFrame(results).T 

# Display the DataFrame
summary_df_100_monthly

In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_100_monthly.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_100_monthly.to_latex(summary_tex_path, index=True, float_format="%.6f")

### 100 Portfolios
#### Annual Regressions

In [ ]:
def regress_annual():
    portfolio_datasets = {
        "100-Portfolios": "100_Portfolios_10x10"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\n[Annual] Processing {label} Data")
        in_sample_results = regressions.run_in_sample_pls_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            end_date='1980-01-01'
        )
        forecast_series, actual_series, R2_oos = regressions.run_recursive_forecast_annual(
            dataset_name=dataset_name,
            weighting="BE_FYt-1_to_ME_June_t",
            h=1,
            start_train_year=1930,
            end_train_year=1980,
            end_forecast_year=2010,
            n_components=1
        )
        regressions.display_results_annual(label, in_sample_results, forecast_series, actual_series, R2_oos)

        results[label] = {
            "R2 In-Sample": in_sample_results["third_model"].rsquared,  
            "R2 Out-of-Sample": R2_oos
        }
    
    return results

# Run function and extract summary DataFrame
results_annual = regress_annual()
summary_df_100_annual = pd.DataFrame(results_annual).T

# Display the DataFrame
summary_df_100_annual

In [ ]:
# Define the directory and file path
summary_tex_dir = "../reports/tables"
summary_tex_path = os.path.join(summary_tex_dir, "summary_table_100_annual.tex")

# Ensure the directory exists
os.makedirs(summary_tex_dir, exist_ok=True)

# Export to LaTeX
summary_df_100_annual.to_latex(summary_tex_path, index=True, float_format="%.6f")